In [2]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import applications
from keras import backend as K

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
import keras.backend as K
from keras_tqdm import TQDMNotebookCallback
from keras_tqdm import TQDMCallback

def build_labels_dict(dataset_path, recipe_food_map_path):
    print("[INFO] loading labels ...")
    recipe_food_map = pd.read_csv(recipe_food_map_path)
    labels_list = list(recipe_food_map['category'].unique())
    food=list(recipe_food_map['photo_id'])
    cat=list(recipe_food_map['category'])
    recipe_food_dict = {food[i]: cat[i] for i in range(len(cat))}
    labels_list.sort()
    return recipe_food_dict, labels_list

if __name__=="__main__":

    MODELS_DIR = 'models'
    DATA_DIR = ''
    RECIPE_FOOD_MAP = os.path.join(DATA_DIR, 'sampledata.csv')
    TYPE_CLASSIFIER = 'multiclass'
    TRAIN_DIR = os.path.join(DATA_DIR, 'sample_training')
    VALID_DIR = os.path.join(DATA_DIR, 'sample_testing')
    BATCH_SIZE = 16
    EPOCHS = 100
    INIT_LR = 1e-6
    IMG_WIDTH, IMG_HEIGHT = 299, 299  
    
    if K.image_data_format() == 'channels_first':
        input_shape = (3, IMG_WIDTH, IMG_HEIGHT)
    else:
        input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)

    num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

    num_train_steps = num_train_samples // BATCH_SIZE + 1
    num_valid_steps = num_valid_samples // BATCH_SIZE + 1

    recipe_food_dict, labels_list = build_labels_dict(DATA_DIR, RECIPE_FOOD_MAP)
    print ("Number of labels {}".format(len(labels_list)))

    # construct the image generator for data augmentation
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       rotation_range=25,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode="nearest")
    test_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')
    validation_generator = test_datagen.flow_from_directory(VALID_DIR, target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')

    label_map = (train_generator.class_indices)
    label_map = dict((v, k) for k, v in label_map.items())
    print(label_map)

    # create the base pre-trained model
    base_model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # let's add a fully-connected layer
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)


    predictions = Dense(train_generator.num_classes, activation='softmax')(x)


    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # compile the model using binary cross-entropy rather than categorical cross-entropy -- th==may seem counterintuitive for
    # multi-label classification, but keep in mind that the goal here ==to treat each output label as an independent Bernoulli distribution:
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=INIT_LR), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    early_stopping = EarlyStopping(patience=15)

    checkpointer = ModelCheckpoint(os.path.join(MODELS_DIR, 'inceptionv3_' + TYPE_CLASSIFIER + '_best.h5'), verbose=1, save_best_only=True)

    # train the network
    print("[INFO] training network...")
    history = model.fit(train_generator, steps_per_epoch=num_train_steps, epochs=EPOCHS, verbose=1, callbacks=[early_stopping, checkpointer], validation_data=validation_generator, validation_steps=num_valid_steps)
    model.save(os.path.join(MODELS_DIR, 'inceptionv3_' + TYPE_CLASSIFIER + '_final.h5'))


[INFO] loading labels ...
Number of labels 11
Found 1650 images belonging to 11 classes.
Found 550 images belonging to 11 classes.
{0: 'Bread', 1: 'Dairy product', 2: 'Dessert', 3: 'Egg', 4: 'Fried food', 5: 'Meat', 6: 'Noodles-Pasta', 7: 'Rice', 8: 'Seafood', 9: 'Soup', 10: 'Vegetable-Fruit'}
[INFO] training network...
Epoch 1/100
104/104 [==============================] - 974s 9s/step - loss: 2.5507 - categorical_accuracy: 0.0925 - val_loss: 2.4439 - val_categorical_accuracy: 0.1091

Epoch 00001: val_loss improved from inf to 2.44386, saving model to models\inceptionv3_multiclass_best.h5
Epoch 2/100
104/104 [==============================] - 989s 9s/step - loss: 2.4774 - categorical_accuracy: 0.1052 - val_loss: 2.3730 - val_categorical_accuracy: 0.1455

Epoch 00002: val_loss improved from 2.44386 to 2.37305, saving model to models\inceptionv3_multiclass_best.h5
Epoch 3/100
104/104 [==============================] - 1043s 10s/step - loss: 2.4138 - categorical_accuracy: 0.1257 - val_lo

104/104 [==============================] - 997s 10s/step - loss: 1.1915 - categorical_accuracy: 0.6737 - val_loss: 1.1238 - val_categorical_accuracy: 0.7073

Epoch 00029: val_loss improved from 1.15687 to 1.12376, saving model to models\inceptionv3_multiclass_best.h5
Epoch 30/100
104/104 [==============================] - 1073s 10s/step - loss: 1.1771 - categorical_accuracy: 0.6770 - val_loss: 1.0901 - val_categorical_accuracy: 0.7109

Epoch 00030: val_loss improved from 1.12376 to 1.09007, saving model to models\inceptionv3_multiclass_best.h5
Epoch 31/100
104/104 [==============================] - 978s 9s/step - loss: 1.1377 - categorical_accuracy: 0.6830 - val_loss: 1.0609 - val_categorical_accuracy: 0.7218

Epoch 00031: val_loss improved from 1.09007 to 1.06095, saving model to models\inceptionv3_multiclass_best.h5
Epoch 32/100
104/104 [==============================] - 949s 9s/step - loss: 1.0976 - categorical_accuracy: 0.7038 - val_loss: 1.0334 - val_categorical_accuracy: 0.7309



104/104 [==============================] - 941s 9s/step - loss: 0.6151 - categorical_accuracy: 0.8352 - val_loss: 0.5967 - val_categorical_accuracy: 0.8327

Epoch 00058: val_loss improved from 0.60480 to 0.59673, saving model to models\inceptionv3_multiclass_best.h5
Epoch 59/100
104/104 [==============================] - 940s 9s/step - loss: 0.5827 - categorical_accuracy: 0.8379 - val_loss: 0.5860 - val_categorical_accuracy: 0.8327

Epoch 00059: val_loss improved from 0.59673 to 0.58599, saving model to models\inceptionv3_multiclass_best.h5
Epoch 60/100
104/104 [==============================] - 941s 9s/step - loss: 0.6332 - categorical_accuracy: 0.8114 - val_loss: 0.5772 - val_categorical_accuracy: 0.8309

Epoch 00060: val_loss improved from 0.58599 to 0.57722, saving model to models\inceptionv3_multiclass_best.h5
Epoch 61/100
104/104 [==============================] - 944s 9s/step - loss: 0.5972 - categorical_accuracy: 0.8122 - val_loss: 0.5692 - val_categorical_accuracy: 0.8364

Epo

KeyboardInterrupt: 

In [ ]:
model.save(os.path.join(MODELS_DIR, 'inceptionv3_' + TYPE_CLASSIFIER + '_final.h5'))
show_acc_history(history)
show_loss_history(history)

